#####**UNIVERSIDAD AUTONOMA DE CHIHUAHUA**
#####**FACULTAD DE INGENIERIA**
#####**SHAKESPEARE**
#####**NOMBRE:** JOHANN LOZANO ENRIQUEZ
#####**GRUPO:** 8CC2
#####**MATRICULA:** 338834
#####**DOCENTE:** JESUS ROBERTO LOPEZ SANTILLAN
#####**MATERIA:** DATA SCIENCE
#####***Chihuahua, Chih. 28 de Noviembre del 2023***

In [ ]:
from tensorflow import keras

shakespeare_url = "https://homl.info/shsakepeare" # shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [ ]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [ ]:
max_id = len(tokenizer.word_index) # number of distinct characters
#dataset_size = tokenizer.document_count # total number of character
dataset_size = sum([x for _,x in tokenizer.word_counts.items()])
print(max_id)
print(dataset_size)
print(type(dataset_size))

39
1115394
<class 'int'>


In [ ]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [ ]:
import tensorflow as tf
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

In [ ]:
dataset = dataset.prefetch(1)

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [ ]:
"""model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history=model.fit(dataset, epochs=20)
model.save('my_model.h5')"""

'model = keras.models.Sequential([\n    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],dropout=0.2),\n    keras.layers.GRU(128, return_sequences=True,\n                     #dropout=0.2, recurrent_dropout=0.2),\n                     dropout=0.2),\n    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))\n])\nmodel.compile(loss="sparse_categorical_crossentropy", optimizer="adam")\nhistory=model.fit(dataset, epochs=20)\nmodel.save(\'my_model.h5\')'

In [ ]:
model = tf.keras.models.load_model('my_model.h5')

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, None, 128)         64896     
                                                                 
 gru_3 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed_1 (TimeDi  (None, None, 39)          5031      
 stributed)                                                      
                                                                 
Total params: 168999 (660.15 KB)
Trainable params: 168999 (660.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) - 1
  return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred_prob = model.predict(X_new)
Y_pred_classes = np.argmax(Y_pred_prob, axis=-1)

predicted_text = tokenizer.sequences_to_texts(Y_pred_classes +1)[0][-1]
print(predicted_text)

1/1 [==============================] - 7s 7s/step
u


In [ ]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

In [ ]:
print(complete_text("t", temperature=0.2))

1/1 [==============================] - 0s 21ms/step
the streeds,
and therefore let him go to her to her


In [ ]:
tf.random.set_seed(42)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
  dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
  dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_length))
  datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)

In [ ]:
stateful_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2,batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [ ]:
stateful_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
stateful_model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 14s 23ms/step - loss: 2.6301
Epoch 2/50
313/313 [==============================] - 5s 16ms/step - loss: 2.2366
Epoch 3/50
313/313 [==============================] - 6s 19ms/step - loss: 2.1060
Epoch 4/50
313/313 [==============================] - 7s 23ms/step - loss: 2.0305
Epoch 5/50
313/313 [==============================] - 7s 21ms/step - loss: 1.9805
Epoch 6/50
313/313 [==============================] - 7s 21ms/step - loss: 1.9419
Epoch 7/50
313/313 [==============================] - 5s 16ms/step - loss: 1.9152
Epoch 8/50
313/313 [==============================] - 6s 18ms/step - loss: 1.8934
Epoch 9/50
313/313 [==============================] - 7s 22ms/step - loss: 1.8772
Epoch 10/50
313/313 [==============================] - 5s 16ms/step - loss: 1.8601
Epoch 11/50
313/313 [==============================] - 7s 23ms/step - loss: 1.8489
Epoch 12/50
313/313 [==============================] - 5s 16ms/step - loss: 1.8371
Epoch 13/50


In [ ]:
stateful_model.save('my_model_stateful.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [ ]:
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model
tf.random.set_seed(42)
print(complete_text("t"))

1/1 [==============================] - 0s 33ms/step
thing idder whom
either the slave. but i have stone
